In [2]:
import pandas as pd
import numpy as np
import cv2
import psycopg2 as pg
import dlib
with open('credentials.txt') as creds:
    db,user,host,port = creds.read().splitlines()
connect = pg.connect(dbname=db, user=user, host=host, port=port)
cursor = connect.cursor()

In [3]:
cursor.execute('SELECT item_name FROM items')

In [4]:
food_items = [tpl[0] for tpl in cursor.fetchall()] 
# названия блюд

In [5]:
cursor.execute("""
SELECT t.tag_name, array_agg(i.item_name) FROM items_tags it, items i, tags t 
WHERE it.item_id = i.item_id AND it.tag_id = t.tag_id GROUP BY t.tag_name;
""")

In [6]:
menu = dict(cursor.fetchall())
print(menu) # в каких блюдах встречается тот или иной ингредиент

{'Sprite': ['Sprite'], 'Big Mac Sauce': ['Big Mac'], 'Main Dish': ['Chicken McNuggets', 'Artisan Grilled Chicken Sandwich', 'Big Mac', 'Buttermilk Crispy Chicken Sandwich', 'Buttermilk Crispy Tenders', 'Cheeseburger', 'Double Cheeseburger', 'Double Quarter Pounder with Cheese', 'Filet-O-Fish', 'Hamburger', 'McChicken', 'McDouble', 'Quarter Pounder with Cheese', 'Quarter Pounder with Cheese Deluxe'], 'Roll': ['Sausage Burrito'], 'Hotcakes': ['Big Breakfast with Hotcakes', 'Hotcakes', 'Hotcakes and Sausage'], 'Hotcake Syrup': ['Big Breakfast with Hotcakes', 'Hotcakes', 'Hotcakes and Sausage'], 'Butter': ['Bacon, Egg  & Cheese Biscuit', 'Bacon, Egg & Cheese Bagel', 'Bacon, Egg & Cheese McGriddles', 'Big Breakfast', 'Big Breakfast with Hotcakes', 'Big Breakfast with Hotcakes', 'Egg McMuffin', 'Hotcakes', 'Hotcakes and Sausage', 'Sausage Biscuit', 'Sausage Biscuit  with Egg', 'Sausage McGriddles', 'Sausage McMuffin', 'Sausage McMuffin with Egg', 'Sausage McMuffin with Egg', 'Sausage, Egg & 

In [7]:
genderNet = cv2.dnn.readNet("gender_net.caffemodel", "gender_deploy.prototxt")

genderList = ['Male','Female']
ageList = np.arange(0, 101).reshape(101, 1)

ageNet = cv2.dnn.readNet("age_net.caffemodel", "age_deploy.prototxt")
detector = dlib.get_frontal_face_detector() # модель для поиска лица
sp = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat') # модель для поиска глаз и носа на фото
facerec = dlib.face_recognition_model_v1( 'dlib_face_recognition_resnet_model_v1.dat') #

ageList = ['(0 - 2)', '(4 - 6)', '(8 - 12)', '(15 - 20)', '(25 - 32)', '(38 - 43)', '(48 - 53)', '(60 - 100)']


In [7]:
for age in ageList:
    a = (age,)
    cursor.execute("INSERT INTO age_groups(age_group) VALUES(%s)",a)

connect.commit()

In [8]:
def process(path):
    im=cv2.imread(path)
    rgb_image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    dets = detector(rgb_image)
    if dets:
        shape = sp(rgb_image, dets[0])
        descriptor = list(facerec.compute_face_descriptor(rgb_image, shape)) # по особым точкам лица составляем вектор
        blob = cv2.dnn.blobFromImage(rgb_image, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False, crop=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderPreds[0].argmax()

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = agePreds[0].argmax()

        return descriptor, age, gender
    return None


In [9]:
path='lfw-deepfunneled/'

In [ ]:
users = []
from tqdm import tqdm
import glob
people = glob.glob(path+'/*')
people.sort()
i=1
for person in tqdm(people,total=len(people)):
    
    p_imgs = glob.glob(person+'/*') 
    genders=[]
    ages=[]
    descs=[]
    for im in p_imgs:
        
        res = process(im)
        if res is not None:
            desc,age,gender=res
            descs.append(desc)
            ages.append(age)
            genders.append(gender)
    try:       
        desc = np.mean(descs,axis=0)
        gender = int(np.round(np.mean(genders)))
        age = bool(int(np.round(np.mean(ages))))
    except:
        print (gender,age,person)
    
    #users.append([i,gender,age,desc])
    i+=1
    

 15%|█▍        | 854/5749 [02:48<15:54,  5.13it/s]  /Users/eyentei/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/eyentei/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 True lfw-deepfunneled/Chan_Ho_Park


 19%|█▉        | 1113/5749 [03:52<11:45,  6.57it/s]  

1 True lfw-deepfunneled/Damarius_Bilbo


In [1]:
orders=[] 
orders_items=[] 

# сгенерируем 10000 заказов
for i in range(1,10001):
    user = users[np.random.randint(n_users)]
    user_id = user[0]
    gender = user[1]
    vector = user[2]
    
    dishes = set()
        
    p = 0 #уменьшаем вероятность заказать основное блюдо на последующих итерациях
    for j in range(3):
        main_types = ['Breakfast', 'Beef', 'Chicken & Fish'] # категории главных блюд
        is_main_picked = [main_types, None]
        main_choice = np.random.choice(is_main_picked, p=[0.8-p, 0.2+p]) # будет ли в заказе главное блюдо?

        if main_choice:
            main_type = np.random.choice(main_types, p=[0.2, 0.60, 0.2]) # если да, то какое? 

            if main_type == 'Breakfast':
                # вручную заданные вероятности заказа блюд по полу, далее так же
                m = [0.08,0.07,0.07,0.15,0.1,0.05,0.05,0.02,0.02,0.02,0.05,0.05,0.07,0.04,0.05,0.06,0.05]
                f = [0.05,0.05,0.05,0.05,0.05,0.08,0.22,0.1,0.08,0.05,0.04,0.03,0.02,0.04,0.04,0.03,0.02]
                
                if gender == 1:
                    dishes.add(np.random.choice(menu['Breakfast'], p = m))
                elif gender == 0:
                    dishes.add(np.random.choice(menu['Breakfast'], p = f))
                
            elif main_type == 'Beef':
                m = [0.25,0.05,0.1,0.15,0.05,0.13,0.12,0.15]
                f = [0.05,0.2,0.15,0.1,0.2,0.1,0.1,0.1]

                if gender == 1:
                    dishes.add(np.random.choice(menu['Beef'], p = m))
                elif gender == 0:
                    dishes.add(np.random.choice(menu['Beef'], p = f))

            elif main_type == 'Chicken & Fish':
                prob=[0.2,0.1,0.04,0.04,0.1,0.2,0.2,0.04,0.04,0.04]
                dishes.add(np.random.choice(menu['Chicken']+menu['Fish'],p=prob))
        

        side_types = ['Salad','Snack']
        is_side_picked = [side_types, None]
        side_choice = np.random.choice(is_side_picked, p=[0.6-p, 0.4+p]) # будет ли в заказе салат/картошка?

        if side_choice:
            # также для мужчин и женщин разные вероятности заказа того или иного блюда
            if gender == 1:
                side_type = np.random.choice(side_types, p=[0.15, 0.85]) 

            elif gender == 0:
                side_type = np.random.choice(side_types, p=[0.35, 0.65])  

            if side_type == 'Salad':
                prob = [0.25,0.25,0,0.25,0.25]
                dishes.add(np.random.choice(menu['Salad'],p=prob))

            elif side_type == 'Snack':
                prob=[0.025,0.025,0.025,0.9,0.025]
                dishes.add(np.random.choice(menu['Snack'], p = prob))


        dessert_types = ['Dessert']
        is_dessert_picked = [dessert_types, None]
        dessert_choice = np.random.choice(is_dessert_picked, p=[0.6-p, 0.4+p]) # будет ли в заказе десерт?
        if dessert_choice:
            m = [0.03,0.05,0.1,0.1,0.02,0.1,0.1,0.01,0.05,0.05,0.01,0.01,0.01,0.05,0.01,0.05,0.05,0.1,0.1]
            f = [0.03,0.01,0.05,0.02,0.08,0.07,0.07,0.07,0.04,0.04,0.03,0.03,0.03,0.04,0.07,0.1,0.1,0.06,0.06]
            if gender == 1:
                dishes.add(np.random.choice(menu['Dessert'], p = m))
            elif gender == 0:
                dishes.add(np.random.choice(menu['Dessert'], p = f))


        drink_types = ['Hot Beverage', 'Cold Beverage']
        is_drink_picked = [drink_types, None]
        drink_choice = np.random.choice(is_drink_picked, p=[0.9-p, 0.1+p])  # будет ли в заказе напиток?

        if drink_choice:

            drink_type = np.random.choice(drink_types, p=[0.5, 0.5])  

            if drink_type == 'Hot Beverage':
                 dishes.add(np.random.choice(menu['Hot Beverage']))

            elif drink_type == 'Cold Beverage':
                 dishes.add(np.random.choice(menu['Cold Beverage']))

        
    order = {'order_id':i,'user_id':user_id} # создаем заказ 
    orders.append(order)
    
    for d in dishes:
        order_item = {'order_id':i, 'item_id':food_items.index(d)} # сопоставляем заказ и продукты
        orders_items.append(order_item)
            

NameError: name 'users' is not defined

In [8]:
orders = pd.DataFrame(orders).fillna(0)

In [9]:
orders_items = pd.DataFrame(orders_items).fillna(0)

In [10]:
orders.head()

,order_id,user_id
0,1,640
1,2,737
2,3,1332
3,4,2018
4,5,1080


In [11]:
orders_items.head()

,item_id,order_id
0,63,1
1,24,1
2,16,1
3,13,1
4,89,1


In [13]:
# заполняем таблицу пользователей сгенерированными данными
for user in users:
    data = (user[2],user[1])
    cursor.execute("INSERT INTO users(vector, gender) VALUES(cube(%s),%s)",data)

connect.commit()

In [34]:
# заполняем таблицу заказов сгенерированными данными

for _, order in orders.iterrows():  
    data = (int(order.user_id),)
    cursor.execute("INSERT INTO orders(user_id) VALUES(%s)", data)
connect.commit()

In [40]:
for _,order in orders_items.iterrows():
    data = (int(order.order_id), int(order.item_id+1))
    cursor.execute("INSERT INTO orders_items(order_id, item_id) values (%s, %s)", data)
    
connect.commit() 

In [43]:
connect.close()